In [22]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn



## Components of training a model
training a model can be described like:<br>
    1.we get data -pairs of questions and answers<br>
    2.for a pair(x,y) we run x through model to get the model's answer y hat<br>
    3.then a teacher gives a model a grade based on how wrong y hat is<br>
    4.based on the grade, we figurte out who;s fault the error is<br>
    5.then we fix the fault so the model can do better next tiem<br>
    

Dataset class is used to process data and load single pairs (x,y)<br>
Data loader class uses your dataset class to get single pairs and group them into batches<br>

In [4]:
class MyDataset(data.Dataset):
    def __init__(self,X,Y):
        """
        use __init__ to load the data to the class
        or process so it can be access later
        """
        self.X = X
        self.Y = Y
    
    def __len__(self):
        """"
        pytorch will use __len__ to know how many (x,y)
        pairs are in your dataset
        """
        return len(self.Y)
    
    def __getitem__(self,index):
        """
        After using len to get how many sampels there are, pytorch will use 
        _getitem__ to ask for certain sample,so getitem(i) should return the ith
        sample,you should use igtem to do some final processing before it's sent out
        caution: getitem will be called many times so you should do as little work in
        this step and try to keep heavy work at __init__ since it is only called once
        """
        X = self.X(index).float().reshape(-1) #flatten the input
        Y = self.Y(index).long()
        return X,Y
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-4c6c48f6d8bf>, line 13)

In [11]:
#pytorch model更多是看图说话
class our_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(3,4)
        self.layer2 = nn.Linear(4,4)
        self.layer3 = nn.Linear(4,1)
    def forward(self,x):
        """
        remember each nn.Linear has trainable parameters, by calling
        parameters() on your mode you will see theweight and bias of each layer
        thanks to the magic backend
        """
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        return out
    

In [15]:
model = our_model()
print(model)

our_model(
  (layer1): Linear(in_features=3, out_features=4, bias=True)
  (layer2): Linear(in_features=4, out_features=4, bias=True)
  (layer3): Linear(in_features=4, out_features=1, bias=True)
)


after you have the model, you need to put the model on device and
set the model mode, either train or eval

### Before the model training, set model.train() to tell the model save gradients 
### when validating or testing, set model.eval() to tell the model doesn't need save gradient(save time and memory)

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 3e-4) #we initiaze the

In [27]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

NameError: name 'DataLoader' is not defined

In [26]:
##training
for epch in range(num_epochs):
    model.train()
    for (x,y) in train_loader:
        """
        what is zero grad?
        every call to backward saves gradient  for each paramters in the model
        however call optimizer.step() does not delete the gradients after using them,
        so you want to remove them before you hading the gtradients of next sample
        """
        optimizer.zero_grad()
        
        x = x.to(device)
        y = y.to(device)
        #forward
        output = model(x)
        loss = criterion(output,y)
        #backward
        loss.backward()
        # by doing optimizer step, we update the weights of the model
        # by using the computed gradients
        optimizer.step()
        
        """
        To summarize
        we run x through model
        we compute the loss
        we call loss backward
        we step the optimizer
        then in this loop or next, we zero out the gradient
        """
        

NameError: name 'train_loader' is not defined

### After here, you would generally perform validation (after every epoch or a couple), to see how your model performs on data it is not trained on.
### Validation follows a similar format as training, but without loss.backward() or optimizer.step(). You should check the notebooks for more guidance.